In [2]:
import os

In [6]:
!git clone https://github.com/ifzhang/ByteTrack.git

Cloning into 'ByteTrack'...


In [5]:
!git clone https://github.com/roboflow/supervision.git
!cd supervision

# setup python environment and activate it
!python3 -m venv venv
!source venv/bin/activate

# headless install
!pip install -e "."

# desktop install
!pip install -e ".[desktop]"

Cloning into 'supervision'...
'python3' is not recognized as an internal or external command,
operable program or batch file.
'source' is not recognized as an internal or external command,
operable program or batch file.
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: C:\Users\Aselsan\Desktop\YOLO+BYTETrack
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: C:\Users\Aselsan\Desktop\YOLO+BYTETrack


In [7]:
# Clone the ultralytics repository
!git clone https://github.com/ultralytics/ultralytics

# Navigate to the cloned directory
!cd ultralytics

# Install the package in editable mode for development
!pip install -e .


Cloning into 'ultralytics'...
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: C:\Users\Aselsan\Desktop\YOLO+BYTETrack


In [ ]:
!pip install supervision

In [ ]:
!pip install ultralytics

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import time
from collections import defaultdict
import supervision as sv
from IPython.display import display, Javascript
from google.colab.output import eval_js
import tensorflow as tf
import uuid

In [ ]:
model = YOLO('yolov8m.pt') # You can choose either 'yolov8#' #--> 'n,s,l,x'

In [ ]:
#set the input & output path
input_video_path = "/content/drive/MyDrive/inputs/football.mp4"
output_video_path = "/content/drive/MyDrive/outputs/real_time_.mp4"


In [ ]:

#created video capture object
cap = cv2.VideoCapture(0)

# Get video properties (frame width, frame height, frame rate, etc.)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))


In [ ]:
# Assuming you already have a video capture object 'cap'
frame_width = cap.get(3)
frame_height = cap.get(4)

print(f"Input frame size: {frame_width}x{frame_height}")

In [ ]:
# Define the codec and initialize the video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # You can choose other codecs as needed
desired_fps = 6
out = cv2.VideoWriter(output_video_path, fourcc, desired_fps, (640, 480))

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "Status:";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '' +
          'When finished, click here or on the video to stop this demo';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')


  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
def js_to_frame(js_reply):
    """
    Params:
        js_reply: JavaScript object containing an image from the webcam
    Returns:
        frame: OpenCV BGR image
    """
    # Decode base64 image
    image_bytes = b64decode(js_reply.split(',')[1])
    # Convert bytes to numpy array
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    # Decode numpy array into OpenCV BGR image
    frame = cv2.imdecode(jpg_as_np, flags=1)

    return frame

In [ ]:
def frame_to_js(frame, quality=0.8):
    """
    Params:
        frame: OpenCV BGR image
        quality: Quality for JPEG encoding (default is 0.8)
    Returns:
        js_reply: JavaScript object containing base64-encoded image data
    """
    # Encode the frame as JPEG
    _, buffer = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), int(quality * 100)])

    # Convert the buffer to bytes and then to base64
    jpg_as_bytes = buffer.tobytes()
    jpg_as_base64 = b64encode(jpg_as_bytes).decode('utf-8')

    # Construct the JavaScript object
    js_reply = 'data:image/jpeg;base64,' + jpg_as_base64

    return js_reply

In [ ]:
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
  return bbox_bytes

In [ ]:
#create an object for drawing bboxes

box_annotater = sv.BoxAnnotator(
thickness=2,
text_thickness=1,
text_scale=1,
)
track_history = defaultdict(lambda: [])
tracker = sv.ByteTrack() #init tracker object



## Input File - Output File

In [ ]:
while cap.isOpened(): #start input stream
    ret, frame = cap.read() # read the video frame by frame

    if not ret: # if there is not return value end the loop
        break
    results = model(frame)[0] #detect objects in the frame by using yolov8 model
    detections = sv.Detections.from_yolov8(results) # save the detections detection object structure -> xyxy(bboxes), mask, confidance, class_id, track_id

    detections = tracker.update_with_detections(detections=detections) #run the BYTETrack algorithm

    labels = [
         f"id:{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
         for _, _, confidence, class_id, tracker_id
         in detections
     ] # create label text for bboxes

    frame = box_annotater.annotate(scene=frame, detections=detections, labels=labels) # use the drawing object for draw bboxes with labels on the frame

    out.write(frame) # write the output frame into output path


cap.release() #release the resources
out.release() #release the resources


## Real-Time Input / Real-Time Output 

In [ ]:
from IPython.display import Image
from IPython.display import display, HTML, clear_output

# start streaming video from webcam
# show the final output streaming video to notebook
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frameq = js_to_frame(js_reply["img"])
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    results = model(frameq)[0]
    detections = sv.Detections.from_ultralytics(results)

    detections = tracker.update_with_detections(detections=detections)
    labels = [
         f"id:{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
         for _, _, confidence, class_id, tracker_id
         in detections
     ] # create label text for bboxes
    
    # I can't pass the output frame into real time video stream so I did different approach 
    frame = box_annotater.annotate(scene=frameq, detections=detections, labels=labels)
    for bbox in detections.xyxy:
        x1, y1, x2, y2 = map(int, bbox)
        bbox_array = cv2.rectangle(bbox_array, (x1, y1), (x2, y2), (0, 0, 255), 2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

cap.release() #release the resources
out.release() #release the resources

## Real-Time Input - File Output

In [ ]:
from IPython.display import Image
from IPython.display import display, HTML, clear_output

# start streaming video from webcam
# show the final output streaming video to notebook
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frameq = js_to_frame(js_reply["img"])
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    results = model(frameq)[0] #detect objects in the frame by using yolov8 model
    detections = sv.Detections.from_yolov8(results) # save the detections detection object structure -> xyxy(bboxes), mask, confidance, class_id, track_id

    detections = tracker.update_with_detections(detections=detections)  #run the BYTETrack algorithm
    labels = [
         f"id:{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
         for _, _, confidence, class_id, tracker_id
         in detections
     ] # create label text for bboxes

    frame = box_annotater.annotate(scene=frameq, detections=detections, labels=labels)  # use the drawing object for draw bboxes with labels on the frame
    out.write(frame) # write the output frame into output path

cap.release() #release the resources
out.release() #release the resources